In [22]:
import dotenv
import glob
from IPython.display import Markdown
from IPython.display import display
import os
import pprint

import boto3

In [23]:
print(boto3.__version__)

1.40.21


In [24]:
dotenv.load_dotenv()
CLAUDE_MODEL_ID = os.getenv("BEDROCK_CLAUDE_4_5_MODEL_ID")
CLAUDE_CLIENT = boto3.client("bedrock-runtime",
                             region_name="ap-northeast-1")
TARGET_CODES_FILE_PATH = r"./target_code_folder_for_make_marp_slide/*.py"

USER_PROMPT = (
    "これはpythonプログラムになります。このプログラムの内容を説明するための数枚のスライドをMarkdownで出力してください。\n"
    "適宜、図やフロー等をmermaid記述で盛り込んでも構いません。\n"
    "行っている内容にフォーカスを当ててください。所感や考察などは盛り込まないでください。\n\n"
    "# コード\n"
    "{a}"
)

USER_PROMPT_2 = (
    "これはMarkdownプログラムになります。この内容を分かりやすく説明する形で数枚のスライドのMarkdownに作り直して出力してください。\n"
    "適宜、図やフロー等をmermaid記述で盛り込んでも構いません。\n"
    "記述されている内容にフォーカスを当ててください。所感や考察などは盛り込まないでください。\n\n"
    "# コード\n"
    "{a}"
)

SYSTEM_PROMPT = (
    "あなたはpythonプログラムの内容を分かりやすく説明するスライドをMarkdownで記述する専門家です。\n"
    "同一のプロジェクトで用いられている複数のpythonプログラムのファイルパスがリスト形式で与えられるので、\n"
    "以下のツールを適切に使い分けて、このプロジェクトのpythonプログラムを説明するスライドを作成してください。\n\n"
    "# ツール\n"
    "1. input_data_for_mcp_server_tool_to_make_markdown\tプログラムの内容を把握して、その内容を説明するMarkdownを記述する時に使います。\n"
    "2. input_data_for_complete\tユーザが処理の完了を伝えてきた時に使います。\n\n"
    "# ファイルパス(リスト形式)\n"
    "{a}"
)

MCP_CLIENT_TOOL_SPEC_1_NAME = "input_data_for_mcp_server_tool_to_make_markdown"
MCP_CLIENT_TOOL_SPEC_1_DESCRIPTION = (
    "あなたはMarkdownでスライドを作成する業務に取り組んでいます。"
)
MCP_CLIENT_TOOL_SPEC_1_INPUTSCHEMA_NAME_1 = "target_program_file_path"
MCP_CLIENT_TOOL_SPEC_1_INPUTSCHEMA_DESCRIPTION_1 = (
    "Markdownでスライドを作成するツールのために対象のプログラムのファイルパスを出力します。"
)

MCP_CLIENT_TOOL_SPEC_2_NAME = "input_data_for_mcp_server_tool_to_make_integrated_markdown"
MCP_CLIENT_TOOL_SPEC_2_DESCRIPTION = (
    "あなたは複数のプログラム毎のMarkdownの内容を踏まえて、それらのMarkdownを纏める形で1つのMarkdownとして分かりやすいスライドを作成する業務に取り組んでいます。"
)
MCP_CLIENT_TOOL_SPEC_2_INPUTSCHEMA_NAME_1 = "input_data_to_make_integrated_markdown"
MCP_CLIENT_TOOL_SPEC_2_INPUTSCHEMA_DESCRIPTION_1 = (
    "1つのMarkdownとして作成し直すツールのために、複数のプログラム毎のMarkdownを統合した形で出力します。"
)

MCP_SERVER_TOOL_SPEC_1_NAME = "mcp_server_tool_to_make_markdown"
MCP_SERVER_TOOL_SPEC_1_DESCRIPTION = (
    "あなたはMarkdownを記述する専門家で、Marpでプレゼンテーションのスライドを作成する業務に取り組んでいます。"
)
MCP_SERVER_TOOL_SPEC_1_INPUTSCHEMA_NAME_1 = "markdown_for_marp"
MCP_SERVER_TOOL_SPEC_1_INPUTSCHEMA_DESCRIPTION_1 = (
    "与えられたプログラムの中身の概要を説明するためのスライドを考えて、そのスライドをMarkdownで作成してください。"
)

MCP_SERVER_TOOL_SPEC_2_NAME = "mcp_server_tool_to_make_integrated_markdown"
MCP_SERVER_TOOL_SPEC_2_DESCRIPTION = (
    "あなたは多くの内容が記述されているMarkdownを分かりやい形に記述し直す専門家で、Marpでのプレゼンテーションのスライドを分かりやすい形に作成し直す業務に取り組んでいます。"
)
MCP_SERVER_TOOL_SPEC_2_INPUTSCHEMA_NAME_1 = "integrated_markdown_for_marp"
MCP_SERVER_TOOL_SPEC_2_INPUTSCHEMA_DESCRIPTION_1 = (
    "与えられたMarkdownの内容を把握して、分かりやすい形のスライドとしてMarkdownを記述し直してください。"
)

In [25]:
def structured_output_for_input_data_of_mcp_server_tool(name, description, schema_name, schema_description):
    return {"toolSpec": {"name": name,
                         "description": description,
                         "inputSchema": {"json": {"type": "object",
                                                  "properties": {schema_name: {"type": "string",
                                                                                 "description": schema_description}
                                                                },
                                                  "required": []
                                                 }
                                        }
                        }
           }


def mcp_client_tools(claude_message_dict_list, claude_debug_flag=False):
    claude_response = CLAUDE_CLIENT.converse(modelId=CLAUDE_MODEL_ID,
                                             messages=claude_message_dict_list,
                                             inferenceConfig={"temperature": 0.2},
                                             toolConfig={"tools": [structured_output_for_input_data_of_mcp_server_tool(name=MCP_CLIENT_TOOL_SPEC_1_NAME,
                                                                                                                       description=MCP_CLIENT_TOOL_SPEC_1_DESCRIPTION,
                                                                                                                       schema_name=MCP_CLIENT_TOOL_SPEC_1_INPUTSCHEMA_NAME_1,
                                                                                                                       schema_description=MCP_CLIENT_TOOL_SPEC_1_INPUTSCHEMA_DESCRIPTION_1),
                                                                   structured_output_for_input_data_of_mcp_server_tool(name=MCP_CLIENT_TOOL_SPEC_2_NAME,
                                                                                                                       description=MCP_CLIENT_TOOL_SPEC_2_DESCRIPTION,
                                                                                                                       schema_name=MCP_CLIENT_TOOL_SPEC_2_INPUTSCHEMA_NAME_1,
                                                                                                                       schema_description=MCP_CLIENT_TOOL_SPEC_2_INPUTSCHEMA_DESCRIPTION_1)]
                                                        })
    if claude_debug_flag == True:
        print(claude_response)
    return claude_response


def structured_output_for_mcp_server_tool(name, description, schema_name, schema_description):
    return {"toolSpec": {"name": name,
                         "description": description,
                         "inputSchema": {"json": {"type": "object",
                                                  "properties": {schema_name: {"type": "string",
                                                                               "description": schema_description}
                                                                },
                                                  "required": []
                                                 }
                                        }
                        }
           }


def mcp_server_tool(claude_message_dict_list, tool_def, tool_name, claude_debug_flag=False):
    claude_response = CLAUDE_CLIENT.converse(modelId=CLAUDE_MODEL_ID,
                                             messages=claude_message_dict_list,
                                             inferenceConfig={"temperature": 0.2},
                                             toolConfig={"tools": [tool_def],
                                                         "toolChoice": {"tool": {"name": tool_name}}
                                                        })
    if claude_debug_flag == True:
        print(claude_response)
    return claude_response


def mcp_server_tool_to_make_markdown(data, claude_debug_flag):
    with open(data["suggested_tool_input"]["target_program_file_path"], mode="r", encoding="utf-8") as f:
            string_target_code = f.read()
    mcp_server_claude_message_list = []
    mcp_server_claude_message = {"role": "user",
                                  "content": [{"text": USER_PROMPT.format(a=string_target_code)}]
                                 }
    mcp_server_claude_message_list.append(mcp_server_claude_message)
    mcp_server_claude_response = mcp_server_tool(claude_message_dict_list=mcp_server_claude_message_list,
                                                 tool_def=structured_output_for_mcp_server_tool(name=MCP_SERVER_TOOL_SPEC_1_NAME,
                                                                                                description=MCP_SERVER_TOOL_SPEC_1_DESCRIPTION,
                                                                                                schema_name=MCP_SERVER_TOOL_SPEC_1_INPUTSCHEMA_NAME_1,
                                                                                                schema_description=MCP_SERVER_TOOL_SPEC_1_INPUTSCHEMA_DESCRIPTION_1),
                                                 tool_name=MCP_SERVER_TOOL_SPEC_1_NAME,
                                                 claude_debug_flag=claude_debug_flag)
    return mcp_server_claude_response["output"]["message"]


def mcp_server_tool_to_make_integrated_markdown(data, claude_debug_flag):
    mcp_server_claude_message_list = []
    mcp_server_claude_message = {"role": "user",
                                  "content": [{"text": USER_PROMPT_2.format(a=data["suggested_tool_input"]["input_data_to_make_integrated_markdown"])}]
                                 }
    mcp_server_claude_message_list.append(mcp_server_claude_message)
    mcp_server_claude_response = mcp_server_tool(claude_message_dict_list=mcp_server_claude_message_list,
                                                 tool_def=structured_output_for_mcp_server_tool(name=MCP_SERVER_TOOL_SPEC_2_NAME,
                                                                                                description=MCP_SERVER_TOOL_SPEC_2_DESCRIPTION,
                                                                                                schema_name=MCP_SERVER_TOOL_SPEC_2_INPUTSCHEMA_NAME_1,
                                                                                                schema_description=MCP_SERVER_TOOL_SPEC_2_INPUTSCHEMA_DESCRIPTION_1),
                                                 tool_name=MCP_SERVER_TOOL_SPEC_2_NAME,
                                                 claude_debug_flag=claude_debug_flag)
    return mcp_server_claude_response["output"]["message"]

In [26]:
target_codes_file_path_list = glob.glob(TARGET_CODES_FILE_PATH)

In [27]:
claude_input_output_message_list = []
claude_input_message_start = {"role": "user",
                              "content": [{"text": SYSTEM_PROMPT.format(a=target_codes_file_path_list)}]
                             }
claude_input_output_message_list.append(claude_input_message_start)
claude_client_response = mcp_client_tools(claude_message_dict_list=claude_input_output_message_list,
                                          claude_debug_flag=False)
claude_input_output_message_list.append(claude_client_response["output"]["message"])

In [28]:
while True:
    if claude_client_response["stopReason"] == "tool_use":
        mcp_client_suggested_tool_dict_list = []
        mcp_client_suggested_tool_dict = {}
        for content in claude_client_response["output"]["message"]["content"]:
            if "toolUse" in content.keys():
                mcp_client_suggested_tool_dict = {"tool_use_id": content["toolUse"]["toolUseId"],
                                                  "suggested_tool_name": content["toolUse"]["name"],
                                                  "suggested_tool_input": content["toolUse"]["input"]}
                mcp_client_suggested_tool_dict_list.append(mcp_client_suggested_tool_dict)
            else:
                continue
        mcp_server_tool_result_dict_list = []
        mcp_server_tool_result_dict = {}
        for suggested_tool_dict in mcp_client_suggested_tool_dict_list:
            if suggested_tool_dict["suggested_tool_name"] == "input_data_for_mcp_server_tool_to_make_markdown":
                mcp_server_tool_response = mcp_server_tool_to_make_markdown(data=suggested_tool_dict,
                                                                            claude_debug_flag=False)
                mcp_server_tool_result_dict = {"toolResult": {"toolUseId": suggested_tool_dict["tool_use_id"],
                                                              "content": [{"text": mcp_server_tool_response["content"][0]["toolUse"]["input"]["markdown_for_marp"]}]
                                                             }
                                              }
                mcp_server_tool_result_dict_list.append(mcp_server_tool_result_dict)
            elif suggested_tool_dict["suggested_tool_name"] == "input_data_for_mcp_server_tool_to_make_integrated_markdown":
                mcp_server_tool_response = mcp_server_tool_to_make_integrated_markdown(data=suggested_tool_dict,
                                                                                       claude_debug_flag=False)
                mcp_server_tool_result_dict = {"toolResult": {"toolUseId": suggested_tool_dict["tool_use_id"],
                                                              "content": [{"text": mcp_server_tool_response["content"][0]["toolUse"]["input"]["integrated_markdown_for_marp"]}]
                                                             }
                                              }
                mcp_server_tool_result_dict_list.append(mcp_server_tool_result_dict)
                output_markdown = mcp_server_tool_response["content"][0]["toolUse"]["input"]["integrated_markdown_for_marp"]
            else:
                continue
        mcp_server_message = {"role": "user",
                              "content": mcp_server_tool_result_dict_list}
        claude_input_output_message_list.append(mcp_server_message)
        claude_client_response = mcp_client_tools(claude_message_dict_list=claude_input_output_message_list,
                                                  claude_debug_flag=False)
        # pprint.pprint(claude_client_response)
        claude_input_output_message_list.append(claude_client_response["output"]["message"])
    elif claude_client_response["stopReason"] == "end_turn":
        break
    else:
        break

In [29]:
print(output_markdown)

---
marp: true
theme: default
paginate: true
---

# 時系列データ機械学習プロジェクト
## 5つのPythonプログラムによる統合システム

---

## プロジェクト構成

```mermaid
graph TB
    A[eda.py<br/>データ探索・前処理] --> B[seq2seq_classification.py<br/>Seq2Seq分類]
    A --> C[seq2seq_regression.py<br/>Seq2Seq回帰]
    A --> D[transformer_encoder-decoder.py<br/>Transformer分類]
    A --> E[transformer_encoder.py<br/>Transformer Encoderのみ]
```

---

## 1. eda.py - データ探索と前処理

### 主な機能
- **大容量CSVファイルの分割**
  - train.csvを60,000行ごとに分割
  - 分割ファイル間のデータズレを修正

- **データ準備**
  - 訓練データとテストデータの準備
  - 欠損値の確認
  - 基本統計量の算出
  - データ分布の確認

---

## 2. seq2seq_classification.py - Seq2Seq分類

### アーキテクチャ
- **特徴量選択**: LightGBM (寄与度500以上)
- **モデル**: Attention機構付きSeq2Seq
- **タスク**: 10クラス分類

### 処理フロー
```mermaid
graph LR
    A[過去10,000行] --> B[Encoder<br/>LSTM]
    B --> C[圧縮表現]
    C --> D[Decoder<br/>LSTM + Attention]
    D --> E[次の10,000行を予測]
```

### 最適化技術
- Mixed Precision Training
- 勾配クリッピング

---

## 3. seq2seq_regression.py - Seq2Seq回帰

### アーキテクチャ
- **モデル**: Attent

In [ ]:
from IPython.display import Markdown
from IPython.display import display


display(Markdown(output_markdown))

In [ ]:
pprint.pprint(claude_input_output_message_list)